In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter,defaultdict
from script_helper import plot,plot_freq,Clean_Data, data_alloc, compute_entropy,total,compute_l2,compute_KL_distance,find_max
import os



In [2]:
with open("data/2019-06-27.10_00_59_059.full.Program.json") as f:
    id_name_lookup = {}
    for line in f:
        info = json.loads(line)
        if ("_id" in info) and ("_source" in info):
            id_name_lookup[info["_id"]] = info
            
with open("data/1558445451-entity.json") as f:
    id_name_lookup2 = {}
    for line in f:
        info = json.loads(line)
        if ("_id" in info) and ("_source" in info):
            id_name_lookup2[info["_id"]] = info

In [4]:
path = "data/hailong_data/07_30"
json_data = []
cur_path = os.path.join(path,"session-vrex-tune-2019-07-30-13.json")
with open(cur_path) as json_file:
    for line in json_file:
        json_data.append(json.loads(line))

In [4]:
id_name_lookup['8624945789561359112']

KeyError: '8624945789561359112'

In [99]:
id_name_lookup2['8624945789561359112']

KeyError: '8624945789561359112'

In [7]:
json_data[0][31]

{'tune': [True, 8],
 'keyPress': [False, 0],
 'deviceId': '8101381919424726843',
 'events': [{'channelId': '4929196409534245104',
   'eventType': 'mediaTuneMetrics',
   'programId': '5274183216525729112',
   'deviceId': '8101381919424726843',
   'term': None,
   'tsString': '2019-07-30 13:53:25,777000 GMT',
   'title': 'KTVU Mornings on 2 at 6am',
   'keyInfo': None,
   'ts': 1564494805777,
   'callSign': 'KTVUDT'},
  {'channelId': '4929196409534245104',
   'eventType': 'mediaTuneMetrics',
   'programId': '5274183216525729112',
   'deviceId': '8101381919424726843',
   'term': None,
   'tsString': '2019-07-30 13:53:26,556000 GMT',
   'title': 'KTVU Mornings on 2 at 6am',
   'keyInfo': None,
   'ts': 1564494806556,
   'callSign': 'KTVUDT'},
  {'channelId': None,
   'eventType': 'vrexSearch',
   'programId': None,
   'deviceId': '8101381919424726843',
   'term': 'the hills',
   'tsString': '2019-07-30 13:53:28,172000 GMT',
   'title': None,
   'keyInfo': None,
   'ts': 1564494808172,
   '

In [97]:
for i in range(len(json_data[0])):
    if json_data[0][i]["events"][0]["term"] == "big bang theory":
        print(json_data[0][i])

{'tune': [True, 2], 'keyPress': [False, 0], 'deviceId': '3272968614736212829', 'events': [{'channelId': None, 'eventType': 'vrexSearch', 'programId': None, 'deviceId': '3272968614736212829', 'term': 'big bang theory', 'tsString': '2019-07-30 13:51:51,331000 GMT', 'title': None, 'keyInfo': None, 'ts': 1564494711331, 'callSign': None}, {'channelId': None, 'eventType': 'mediaTuneMetrics', 'programId': '8624945789561359112', 'deviceId': '3272968614736212829', 'term': None, 'tsString': '2019-07-30 13:52:12,600000 GMT', 'title': 'The Big Bang Theory: S5 Ep22 - The Stag Convergence', 'keyInfo': None, 'ts': 1564494732600, 'callSign': ''}, {'channelId': None, 'eventType': 'mediaTuneMetrics', 'programId': '8624945789561359112', 'deviceId': '3272968614736212829', 'term': None, 'tsString': '2019-07-30 13:52:13,205000 GMT', 'title': 'The Big Bang Theory: S5 Ep22 - The Stag Convergence', 'keyInfo': None, 'ts': 1564494733205, 'callSign': ''}], 'rex': [False, 0], 'exitPress': False, 'vrex': [True, 0]}

{'tune': [True, 2], 'keyPress': [False, 0], 'deviceId': '2383160377651870192', 'events': [{'channelId': None, 'eventType': 'vrexSearch', 'programId': None, 'deviceId': '2383160377651870192', 'term': 'big bang theory', 'tsString': '2019-07-30 13:37:01,508000 GMT', 'title': None, 'keyInfo': None, 'ts': 1564493821508, 'callSign': None}, {'channelId': None, 'eventType': 'mediaTuneMetrics', 'programId': '5986458721011009112', 'deviceId': '2383160377651870192', 'term': None, 'tsString': '2019-07-30 13:37:22,176000 GMT', 'title': 'The Big Bang Theory: S9 Ep19 - The Solder Excursion Diversion', 'keyInfo': None, 'ts': 1564493842176, 'callSign': ''}, {'channelId': None, 'eventType': 'mediaTuneMetrics', 'programId': '5986458721011009112', 'deviceId': '2383160377651870192', 'term': None, 'tsString': '2019-07-30 13:37:22,538000 GMT', 'title': 'The Big Bang Theory: S9 Ep19 - The Solder Excursion Diversion', 'keyInfo': None, 'ts': 1564493842538, 'callSign': ''}], 'rex': [False, 0], 'exitPress': False

In [11]:
tmp = defaultdict(lambda: defaultdict(int))
for session in json_data[0]:
    events = session["events"]
    last_vrex_search=-1
    tune = -1
    for idx,event in enumerate(events):
        if event["eventType"] == "vrexSearch":
            last_vrex_search = idx
    if last_vrex_search >=0:
        for idx in range(last_vrex_search,len(events)):
            event = events[idx]
            if event["eventType"] == "mediaTuneMetrics":
                tune = idx
                break
    if last_vrex_search >=0 and tune >= 0:
        query = events[last_vrex_search]["term"]
        to = events[tune]["title"]
        tmp[query][to] += 1
print(len(tmp))
        

18327


In [ ]:
tmp2 = defaultdict(lambda: list)
for session in json_data[0]:
    events = session["events"]
    last_vrex_search=-1
    tune = -1
    for idx,event in enumerate(events):
        if event["eventType"] == "vrexSearch":
            last_vrex_search = idx
    if last_vrex_search >=0:
        for idx in range(last_vrex_search,len(events)):
            event = events[idx]
            if event["eventType"] == "mediaTuneMetrics":
                tune = idx
                break
    if last_vrex_search >=0 and tune >= 0:
        query = events[last_vrex_search]["term"]
        to = events[tune]["title"]
        tmp[query][to] += 1
print(len(tmp))

In [12]:
def count_it(tmp,key):
    return sum(tmp[key].values())
def remap(d):
    a = {}
    for each in d:
        a[each] = count_it(d,each)
    return a

In [13]:
remaped = remap(tmp)

In [14]:
remaped = Counter(remaped)
remaped.most_common(20)
remaped

Counter({'the loud house': 257,
         'animal planet': 1678,
         '90 day fiance': 528,
         'pbs kids': 936,
         'abc': 4508,
         'tbs': 1060,
         'paw patrol': 2479,
         '189': 18,
         '187': 35,
         'disney': 1687,
         'tnt': 881,
         'nbc': 5304,
         'msn': 65,
         'hgtv': 3223,
         'pbs': 627,
         'usa': 1295,
         'the princess and the frog': 4,
         'andy griffith': 33,
         'espn': 3062,
         'toy story': 236,
         'channel': 105,
         'id': 580,
         'metv': 1914,
         'recordings': 3248,
         'henry danger': 1168,
         'the hills': 509,
         'missing 411': 1,
         '10': 125,
         'the bachelorette': 937,
         'hln': 989,
         'family feud': 32,
         'zombies the movie': 1,
         'netflix': 4394,
         '2': 369,
         'the wire': 22,
         'word party': 25,
         'cartoon network': 3206,
         'cnbc': 2753,
         'bravo': 8

In [56]:
tmp["cnn"]

defaultdict(int,
            {'CNN Newsroom With Poppy Harlow and Jim Sciutto': 7684,
             'New Day With Alisyn Camerota and John Berman': 98,
             'The Andy Griffith Show: S4 Ep30 - Barney and Thelma Lou, Phfftt': 2,
             'I Want My 80s': 1,
             'Today 3rd Hour': 9,
             'Good Morning America': 7,
             'Dateline: S1 Ep42 - A Perfect Spot': 1,
             'Paid Programming': 1,
             'Notting Hill': 1,
             'Morning Joe': 1,
             'CNN Tonight With Don Lemon': 4,
             'The Express': 1,
             'MSNBC Live With Stephanie Ruhle': 9,
             'The Closer: S5 Ep3 - Red Tape': 2,
             'NCIS: S6 Ep6 - Murder 2.0': 3,
             'Flea Market Flip: S10 Ep5 - Ugly Ducklings Into Swans': 5,
             "America's Court With Judge Ross - Promzilla & My Art Is Not Trash": 2,
             'Futurama: S2 Ep4 - Fry and the Slurm Factory': 1,
             'Live with Kelly and Ryan': 11,
             'Get

In [63]:
from fractions import Fraction
import math
def compute_entropy(data, key, m, tilt,top = 20):
    data[key].pop("", None)
    if m!="":
        plot_freq(data,key,top,tilt,m)
    data = Counter(dict(data[key]))
    data = data.most_common(top)
    total_count = sum([x[1] for x in data])
    thresh_data = []
    for x in data:
        ratio =Fraction(x[1],total_count)
        # get rid of low percentage options
        if ratio >0.1:
            thresh_data.append(x)
#     print([x[1]/total_count  for x in data])
#     print([x[1]/total_count  for x in thresh_data])
#     print([Fraction(x[1],total_count) *math.log(Fraction(x[1],total_count),2)  for x in data])
    
        
    H = sum([Fraction(x[1],total_count) *math.log(Fraction(x[1],total_count),2)  for x in thresh_data]) * -1
#     print("Entrophy is %f" % (H))
    # penalize channels that have fewer data
#     H += 1.0/total_count if total_count != 0 else 0
    H = H/8.0
    return H
import heapq
def rank2(data, limit = 5,top=5,first=1000):
    heap = []
    for each in data:
        score = compute_entropy(data,each,"",20,top)
        if total(data[each]) <=limit :
            continue
        else:
            heapq.heappush(heap, (score,each))

    return heapq.nsmallest(first,heap)

In [64]:
query2program = tmp
result = rank2(query2program,5,6,53952)

In [65]:
result

[(-0.0, '027'),
 (-0.0, '100'),
 (-0.0, '100% julian edelman'),
 (-0.0, '1031'),
 (-0.0, '1048'),
 (-0.0, '1061'),
 (-0.0, '109'),
 (-0.0, '1097'),
 (-0.0, '1113'),
 (-0.0, '1123'),
 (-0.0, '1127'),
 (-0.0, '1166'),
 (-0.0, '1181'),
 (-0.0, '12 strong'),
 (-0.0, '1205'),
 (-0.0, '1207'),
 (-0.0, '1243'),
 (-0.0, '13 hours'),
 (-0.0, '134'),
 (-0.0, '1400'),
 (-0.0, '1403'),
 (-0.0, '1410'),
 (-0.0, '1418'),
 (-0.0, '1421'),
 (-0.0, '1425'),
 (-0.0, '1427'),
 (-0.0, '1444'),
 (-0.0, '1449'),
 (-0.0, '145'),
 (-0.0, '1455'),
 (-0.0, '1456'),
 (-0.0, '1457'),
 (-0.0, '1459'),
 (-0.0, '1464'),
 (-0.0, '1465'),
 (-0.0, '1470'),
 (-0.0, '1477'),
 (-0.0, '1478'),
 (-0.0, '1480'),
 (-0.0, '149'),
 (-0.0, '1497'),
 (-0.0, '1655'),
 (-0.0, '1773'),
 (-0.0, '1775'),
 (-0.0, '1970'),
 (-0.0, '2001: a space odyssey'),
 (-0.0, '2012'),
 (-0.0, '22 jump street'),
 (-0.0, '267'),
 (-0.0, '271'),
 (-0.0, '274'),
 (-0.0, '301'),
 (-0.0, '324'),
 (-0.0, '331'),
 (-0.0, '354'),
 (-0.0, '363'),
 (-0.0, '41

In [74]:
sessions2 = [name for idx,(_,name) in enumerate(result)]
json_data2 = {"query":sessions2,"programs":[query2program[each] for each in sessions2]}
 

In [10]:
query2program["nbc"]

NameError: name 'query2program' is not defined